In [11]:
from prompt_gen import generate_prompts 

elements_folder = "elements"  # Đường dẫn đến thư mục elements
prompts_output_folder = "prompt_gen"  # Đường dẫn để lưu prompts
prompts_file_paths = generate_prompts(elements_folder, prompts_output_folder)
for prompt_file_path in prompts_file_paths:
    print(f"Generated prompts at {prompt_file_path}")

Generated prompts file at: prompt_gen\elements_0001_prompts.txt
Generated prompts file at: prompt_gen\elements_0001_prompts.txt
Generated prompts at prompt_gen\elements_0001_prompts.txt


In [1]:
import os
import pandas as pd
import csv
import json
import re

In [2]:
raw_sentence_file = 'prompt_gen/elements_0001_prompts.txt'
labels_folder = "labels_with_ids"
elements_folder = 'elements'
subfolder_path = "0001"

In [3]:
def parse_elements(elements_folder,subfolder_path):
    """Parse the elements CSV files to collect the data."""
    element_data = []
    for file in os.listdir(elements_folder):
        if file.endswith(f"{subfolder_path}.csv"):
            file_path = os.path.join(elements_folder, file)
            with open(file_path, mode='r') as csv_file:
                reader = csv.DictReader(csv_file)
                for row in reader:
                    # Strip and convert to lowercase
                    color = row['color'].strip().lower() if row['color'] else ''
                    type = row['type'].strip().lower() if row['type'] else ''
                    gender = row['gender'].strip().lower() if row['gender'] else ''
                    
                    # Only include rows where both color and type are not empty
                    if color and type and gender:
                        element_data.append({
                            'frame_id': int(row['frame_id']),
                            'class_id': int(row['class_id']),
                            'color': color,
                            'type': type,
                            'gender': gender
                        })
    return element_data

In [4]:
element_data = parse_elements(elements_folder, subfolder_path)

In [5]:
with open(raw_sentence_file, 'r') as file:
        raw_sentences = [line.strip() for line in file.readlines()]

In [6]:
raw_sentences

['woman staff',
 'woman in blue uniform',
 'woman customer',
 'man in white shirt',
 'woman',
 'customer in white shirt',
 'man customer',
 'man customer in white shirt',
 'staff in blue uniform',
 'woman in white shirt',
 'woman staff in blue uniform',
 'customer',
 'man',
 'staff',
 'woman customer in white shirt']

In [58]:
# def find_matching_ids(prompt, element_data):
#     prompt_lower = prompt.lower()  # Convert the prompt to lowercase for case-insensitive matching
#     matching_ids = set()  # Use a set to store unique IDs

#     # Mẫu regex để tìm từ "man" và "woman" độc lập
#     pattern_man = r'\bman\b'
#     pattern_woman = r'\bwoman\b'

#     # Duyệt qua từng phần tử trong element_data
#     for element in element_data:
#         cls_id = element['class_id']
#         color = element['color'].lower()
#         type = element['type'].lower()
#         gender = element['gender'].lower()

#         # Tính xem có bao nhiêu điều kiện trong prompt khớp với element_data
#         matches = 0
#         total_conditions = 0

#         # Kiểm tra từng điều kiện
#         if color in prompt_lower:
#             matches += 1
#         if type in prompt_lower:
#             matches += 1
#         # Sử dụng regex để kiểm tra giới tính
#         is_gender_match = False
#         if re.search(pattern_man, prompt_lower) and gender == 'man':
#             matches += 1
#             is_gender_match = True
#         elif re.search(pattern_woman, prompt_lower) and gender == 'woman':
#             matches += 1
#             is_gender_match = True

#         # Nếu giới tính không khớp, bỏ qua phần tử
#         if ('man' in prompt_lower or 'woman' in prompt_lower) and not is_gender_match:
#             continue

#         # Tính tổng điều kiện trong prompt
#         total_conditions += 'man' in prompt_lower or 'woman' in prompt_lower
#         total_conditions += 'customer' in prompt_lower or 'staff' in prompt_lower
#         total_conditions += 'white shirt' in prompt_lower or 'blue uniform' in prompt_lower

#         # So sánh số điều kiện khớp với tổng số điều kiện cần thiết
#         if matches == total_conditions:
#             matching_ids.add(cls_id)

#     return list(matching_ids)

# # Dữ liệu ví dụ
# element_data = [
#     {'frame_id': 0, 'class_id': 1, 'type': 'staff', 'color': 'blue uniform', 'gender': 'woman'},
#     {'frame_id': 0, 'class_id': 2, 'type': 'staff', 'color': 'blue uniform', 'gender': 'woman'},
#     {'frame_id': 0, 'class_id': 3, 'type': 'customer', 'color': 'white shirt', 'gender': 'woman'},
#     {'frame_id': 0, 'class_id': 4, 'type': 'customer', 'color': 'white shirt', 'gender': 'man'},
#     {'frame_id': 1, 'class_id': 1, 'type': 'staff', 'color': 'blue uniform', 'gender': 'woman'},
#     {'frame_id': 1, 'class_id': 2, 'type': 'staff', 'color': 'blue uniform', 'gender': 'woman'},
#     {'frame_id': 1, 'class_id': 3, 'type': 'customer', 'color': 'white shirt', 'gender': 'woman'},
#     {'frame_id': 1, 'class_id': 4, 'type': 'customer', 'color': 'white shirt', 'gender': 'man'}
# ]

# # Ví dụ về prompt
# prompts = ["woman staff", "man", "customer in white shirt", "woman customer"]  #,"staff in blue uniform","man in white shirt"

# for prompt in prompts:
#     matching_ids = find_matching_ids(prompt, element_data)
#     print(f"Prompt: {prompt} -> Matching IDs: {matching_ids}")


Prompt: woman staff -> Matching IDs: [1, 2]
Prompt: man -> Matching IDs: [4, 5]
Prompt: customer in white shirt -> Matching IDs: [3, 4]
Prompt: woman customer -> Matching IDs: [3]
Prompt: staff in blue uniform -> Matching IDs: [1, 2, 5]
Prompt: man in white shirt -> Matching IDs: [4]


In [63]:
import re

# List of connecting words to ignore


def find_matching_ids(prompt, element_data):
    prompt_lower = prompt.lower()  # Convert prompt to lowercase
    prompt_tokens = prompt_lower.split()  # Split prompt into words

    CONNECTING_WORDS = ['in', 'on', 'with', 'at', 'of', 'for', 'to']

    # Remove connecting words from prompt tokens
    prompt_tokens = [token for token in prompt_tokens if token not in CONNECTING_WORDS]

    matching_ids = set()  # Set to store unique matching IDs

    # Regex pattern for "man" and "woman"
    pattern_man = r'\bman\b'
    pattern_woman = r'\bwoman\b'

    # Iterate over each element in element_data
    for element in element_data:
        cls_id = element['class_id']
        matches = 0  # Number of matching conditions
        total_conditions = len(prompt_tokens)  # Total number of conditions in the prompt
        is_valid = True  # To check if the element matches all conditions

        # Iterate over each key-value pair in the element
        for key, value in element.items():
            if key == 'class_id' or key == 'frame_id':  # Skip fields we don't need to check
                continue

            value_lower = str(value).lower()

            # Check gender with regex for "man" and "woman"
            if key == 'gender':
                if re.search(pattern_man, prompt_lower) and value_lower == 'man':
                    matches += 1
                elif re.search(pattern_woman, prompt_lower) and value_lower == 'woman':
                    matches += 1
                total_conditions += 'man' in prompt_lower or 'woman' in prompt_lower

            # Check numeric conditions (e.g., "age 30", "height 180")
            num_match = re.search(rf'{key} (\d+)', prompt_lower)  # Find numeric conditions
            if num_match:
                num_value = num_match.group(1)
                if str(value) == num_value:  # Compare prompt number with element number
                    matches += 1
                else:
                    is_valid = False

            # Check if each token in the prompt matches the value
            for token in prompt_tokens:
                if token in value_lower:
                    matches += 1

        # If all conditions match and no conditions failed, add class_id to results
        if matches == total_conditions and is_valid:
            matching_ids.add(cls_id)

    return list(matching_ids)

# Example data with more fields
element_data = parse_elements(elements_folder, subfolder_path)

# Example prompts
prompts = ["woman staff", "man", "man white shirt", "woman customer", "staff blue uniform", "man in white shirt", "woman in blue uniform"]

for prompt in prompts:
    matching_ids = find_matching_ids(prompt, element_data)
    print(f"Prompt: {prompt} -> Matching IDs: {matching_ids}")


Prompt: woman staff -> Matching IDs: [1, 2]
Prompt: man -> Matching IDs: [4, 5]
Prompt: man white shirt -> Matching IDs: [4]
Prompt: woman customer -> Matching IDs: [3]
Prompt: staff blue uniform -> Matching IDs: [1, 2, 5]
Prompt: man in white shirt -> Matching IDs: [4]
Prompt: woman in blue uniform -> Matching IDs: [1, 2]


In [110]:
print(element_data)

[{'frame_id': 0, 'class_id': 1, 'color': 'blue uniform', 'type': 'staff', 'gender': 'woman'}, {'frame_id': 0, 'class_id': 2, 'color': 'blue uniform', 'type': 'staff', 'gender': 'woman'}, {'frame_id': 0, 'class_id': 3, 'color': 'white shirt', 'type': 'customer', 'gender': 'woman'}, {'frame_id': 0, 'class_id': 4, 'color': 'white shirt', 'type': 'customer', 'gender': 'man'}, {'frame_id': 1, 'class_id': 1, 'color': 'blue uniform', 'type': 'staff', 'gender': 'woman'}, {'frame_id': 1, 'class_id': 2, 'color': 'blue uniform', 'type': 'staff', 'gender': 'woman'}, {'frame_id': 1, 'class_id': 3, 'color': 'white shirt', 'type': 'customer', 'gender': 'woman'}, {'frame_id': 1, 'class_id': 4, 'color': 'white shirt', 'type': 'customer', 'gender': 'man'}, {'frame_id': 2, 'class_id': 1, 'color': 'blue uniform', 'type': 'staff', 'gender': 'woman'}, {'frame_id': 2, 'class_id': 2, 'color': 'blue uniform', 'type': 'staff', 'gender': 'woman'}, {'frame_id': 2, 'class_id': 3, 'color': 'white shirt', 'type': 'c

In [64]:
for raw_sentence in raw_sentences:
            print(raw_sentence)
            matching_ids = find_matching_ids(raw_sentence, element_data)
            print(matching_ids)
            print("-----------------------------")



woman staff
[1, 2]
-----------------------------
woman in blue uniform
[1, 2]
-----------------------------
woman customer
[3]
-----------------------------
man in white shirt
[4]
-----------------------------
woman
[1, 2, 3]
-----------------------------
customer in white shirt
[3, 4]
-----------------------------
man customer
[4]
-----------------------------
man customer in white shirt
[4]
-----------------------------
staff in blue uniform
[1, 2, 5]
-----------------------------
woman in white shirt
[3]
-----------------------------
woman staff in blue uniform
[1, 2]
-----------------------------
customer
[3, 4]
-----------------------------
man
[4, 5]
-----------------------------
staff
[1, 2, 5]
-----------------------------
woman customer in white shirt
[3]
-----------------------------


In [112]:
find_matching_ids('woman staff',element_data)

[1, 2]

In [14]:
csv_file = 'elements/elements_0001.csv'
df = pd.read_csv(csv_file)
fildf = df[df["class_id"]==6]
fildf

,frame_id,class_id,color,action,gender
6,0,6,white shirt,walking,man
12,1,6,white shirt,walking,man
19,2,6,white shirt,walking,man
24,3,6,white shirt,walking,man
31,4,6,white shirt,walking,man
38,5,6,white shirt,walking,man
39,6,6,white shirt,walking,man
44,7,6,white shirt,walking,man
51,8,6,white shirt,walking,man
56,9,6,white shirt,walking,man


In [21]:
frame_data = {}

# Iterate through each CSV file in the elements folder
for file in os.listdir(elements_folder):
    if file.endswith(f"{subfolder_path}.csv"):
        file_path = os.path.join(elements_folder, file)
        
        with open(file_path, 'r') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                print(row)
                frame_number = int(row['frame_id'])
                class_id = int(row['class_id'])

                # Check if the class_id is in matching_ids
                if class_id in matching_ids:
                    if frame_number not in frame_data:
                        frame_data[frame_number] = []
                    frame_data[frame_number].append(class_id)

# Sort class_ids for each frame_number
for frame_number in frame_data:
    frame_data[frame_number] = sorted(frame_data[frame_number])


{'frame_id': '1', 'class_id': '1', 'color': 'white shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '2', 'color': 'pink shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '3', 'color': 'red shirt', 'action': 'walking', 'gender': 'man'}
{'frame_id': '1', 'class_id': '4', 'color': 'blue shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '5', 'color': 'black shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '6', 'color': 'white shirt', 'action': 'walking', 'gender': 'man'}
{'frame_id': '2', 'class_id': '1', 'color': 'white shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '2', 'class_id': '2', 'color': 'pink shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '2', 'class_id': '3', 'color': 'red shirt', 'action': 'walking', 'gender': 'man'}
{'frame_id': '2', 'class_id': '4', 'color': 'blue shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '2', 'class

In [45]:
file_path

'elements\\elements_1.csv'

In [72]:
for row in reader:
    print(row)

ValueError: I/O operation on closed file.

In [22]:
frame_data  

{1: [2], 2: [2], 0: [2]}

In [23]:
def sanitize_filename(prompt):
    """Sanitize the prompt to create a valid filename."""
    sanitized_prompt = re.sub(r'[<>:"/\\|?*\n]', '', prompt)  # Remove invalid characters
    sanitized_prompt = sanitized_prompt.replace(' ', '_')  # Replace spaces with underscores
    sanitized_prompt = sanitized_prompt[:150]  # Optionally truncate to prevent overly long filenames
    return sanitized_prompt


In [24]:
generated_sentences = set()  # Store sentences to avoid repetition
counter = 0

while len(generated_sentences) < 1:
    paraphrased_sentence = "A person dressed in black clothing was involved in inspecting something."
    if paraphrased_sentence not in generated_sentences:
        generated_sentences.add(paraphrased_sentence)
        
        # Prepare JSON data
        json_data = {
            "label": frame_data,
            "ignore": [],
            "video_name": os.path.basename(subfolder_path),  # Use the subfolder name as the video name
            "sentence": paraphrased_sentence,
            "raw_sentence": "A person in a black uniform engaged in checking out."
        }

        # Sanitize filename by removing invalid characters
        sanitized_prompt = sanitize_filename(paraphrased_sentence)

        # Save JSON file
        print(json_data)
    

{'label': {1: [2], 2: [2], 0: [2]}, 'ignore': [], 'video_name': '0003', 'sentence': 'A person dressed in black clothing was involved in inspecting something.', 'raw_sentence': 'A person in a black uniform engaged in checking out.'}


In [25]:
import pandas as pd

def sort_elements_csv(file_path):
    
    # Read the CSV data into a DataFrame
    df = pd.read_csv(file_path)
    
    # Sort the DataFrame by 'frame_id'
    sorted_df = df.sort_values(by='frame_id')
    print(sorted_df)
    

# Usage example
input_file = 'elements/elements_0003.csv'  # Replace with your input file path

sort_elements_csv(input_file)

    frame_id  class_id        color   action gender
88         0         7  white shirt  walking  woman
86         0         5  black shirt  walking  woman
85         0         4   blue shirt  walking  woman
84         0         3    red shirt  walking    man
83         0         1  white shirt  walking  woman
..       ...       ...          ...      ...    ...
77        20         5  black shirt  walking  woman
78        20         8  black shirt  walking  woman
79        21         5  black shirt  walking  woman
80        21         8  black shirt  walking  woman
81        22         8  black shirt  walking  woman

[89 rows x 5 columns]


In [32]:
file_path = 'elements/elements_0003.csv'
# Read the CSV data into a DataFrame
df = pd.read_csv(file_path)

# Sort the DataFrame by 'frame_id'
sorted_df = df.sort_values(by='frame_id')

# Save the sorted DataFrame to a new CSV file
sorted_df.to_csv(file_path, index=False)
print(file_path)

elements/elements_0003.csv


In [30]:
import csv
import os
import random
import pandas as pd
from pathlib import Path
generated_files = []

# Định nghĩa các mẫu đa dạng cho các prompt
templates = [
    "A {gender} with {color} is {type}",
    "A {gender} wearing {color} is {type}",
    "A {gender} dressed in {color} is {type}",
    "A {gender} who has {color} is {type}",
    "A {gender} having {color} is {type}"

    
]

# Lấy danh sách tất cả các file .csv trong folder elements
csv_files = [f for f in os.listdir(elements_folder) if f.endswith('.csv')]
# Xử lý từng file CSV
for elements_file in csv_files:
    prompts = []
    csv_path = os.path.join(elements_folder, elements_file)
    print(csv_path)
    with open(csv_path, mode='r') as file:
        reader = csv.DictReader(file)
        
        for row in reader:
            print(row)
            color = row.get('color', '').strip().lower()
            type = row.get('type', '').strip().lower()
            gender = row.get('gender', '').strip().lower()

            # Chọn ngẫu nhiên một template
            template = random.choice(templates)

            # Điền vào template với dữ liệu từ CSV
            prompt = template.format(color=color, type=type, gender=gender)
            if prompt not in prompts: 
                prompts.append(prompt)

        

   

elements\elements_0003.csv
{'frame_id': '1', 'class_id': '1', 'color': 'white shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '2', 'color': 'pink shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '3', 'color': 'red shirt', 'action': 'walking', 'gender': 'man'}
{'frame_id': '1', 'class_id': '4', 'color': 'blue shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '5', 'color': 'black shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '1', 'class_id': '6', 'color': 'white shirt', 'action': 'walking', 'gender': 'man'}
{'frame_id': '2', 'class_id': '1', 'color': 'white shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '2', 'class_id': '2', 'color': 'pink shirt', 'action': 'walking', 'gender': 'woman'}
{'frame_id': '2', 'class_id': '3', 'color': 'red shirt', 'action': 'walking', 'gender': 'man'}
{'frame_id': '2', 'class_id': '4', 'color': 'blue shirt', 'action': 'walking', 'gender': 'woman